In [4]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import re
from collections import Counter

def analyze_duplicates(df, text_similarity_threshold=0.85):
    """
    Comprehensive duplicate analysis for newspaper data
    """
    print("=== DUPLICATE ANALYSIS ===")
    print(f"Total articles: {len(df)}")
    
    # 1. EXACT DUPLICATES (all fields identical)
    print("\n1. EXACT DUPLICATES:")
    exact_dupes = df.duplicated(subset=['title', 'main_text', 'publication_name', 'year'])
    exact_dupe_count = exact_dupes.sum()
    print(f"   Exact duplicates: {exact_dupe_count}")
    
    if exact_dupe_count > 0:
        print("   Sample exact duplicates:")
        sample_exact = df[exact_dupes][['year', 'title', 'publication_name']].head(3)
        print(sample_exact.to_string(index=False))
    
    # 2. TITLE + YEAR DUPLICATES (same story, possibly different editions)
    print("\n2. SAME TITLE + YEAR (different editions/publications):")
    title_year_dupes = df.duplicated(subset=['title', 'year'], keep=False)
    title_year_dupe_count = title_year_dupes.sum()
    print(f"   Articles with same title+year: {title_year_dupe_count}")
    
    if title_year_dupe_count > 0:
        print("   Sample same title+year:")
        sample_title = df[title_year_dupes][['year', 'title', 'publication_name']].head(6)
        print(sample_title.to_string(index=False))
    
    # 3. EDITION VARIANTS (same title with edition markers)
    print("\n3. EDITION VARIANTS:")
    edition_patterns = [r'\[.*Edition.*\]', r'\[Final\]', r'\[METRO\]', r'\[Early\]']
    
    # Create title without edition markers
    df['title_clean'] = df['title'].astype(str)
    for pattern in edition_patterns:
        df['title_clean'] = df['title_clean'].str.replace(pattern, '', regex=True)
    df['title_clean'] = df['title_clean'].str.strip()
    
    # Find articles with same clean title + year
    clean_title_dupes = df.duplicated(subset=['title_clean', 'year'], keep=False)
    clean_title_dupe_count = clean_title_dupes.sum()
    edition_variants = clean_title_dupe_count - title_year_dupe_count
    print(f"   Edition variants: {edition_variants}")
    
    if edition_variants > 0:
        print("   Sample edition variants:")
        # Show cases where clean title matches but original title doesn't
        mask = clean_title_dupes & ~title_year_dupes
        sample_editions = df[mask][['year', 'title', 'publication_name']].head(4)
        print(sample_editions.to_string(index=False))
    
    # # 4. SIMILAR TITLES (typos, minor differences)
    # print("\n4. SIMILAR TITLES (potential typos/variations):")
    # similar_titles = find_similar_titles(df, similarity_threshold=0.9)
    # print(f"   Similar title pairs: {len(similar_titles)}")
    
    # if similar_titles:
    #     print("   Sample similar titles:")
    #     for i, (title1, title2, similarity) in enumerate(similar_titles[:3]):
    #         print(f"   {i+1}. Similarity: {similarity:.3f}")
    #         print(f"      '{title1}'")
    #         print(f"      '{title2}'")
    
    # 5. WIRE SERVICE ARTICLES (same text, different publications)
    # print("\n5. POTENTIAL WIRE SERVICE ARTICLES:")
    # wire_articles = find_wire_service_articles(df, text_similarity_threshold)
    # print(f"   Potential wire service duplicates: {len(wire_articles)}")
    
    # if wire_articles:
    #     print("   Sample wire service articles:")
    #     for i, article_group in enumerate(wire_articles[:2]):
    #         print(f"   Group {i+1}: {len(article_group)} publications")
    #         sample_group = df.iloc[article_group[:3]]
    #         print(sample_group[['title', 'publication_name']].to_string(index=False))
    #         print()
    
    # 6. PUBLICATION PATTERNS
    print("\n6. PUBLICATION PATTERNS:")
    pub_counts = df['publication_name'].value_counts()
    print(f"   Total unique publications: {len(pub_counts)}")
    print(f"   Top 5 publications by article count:")
    for pub, count in pub_counts.head().items():
        print(f"     {pub}: {count} articles")
    
    # 7. YEARLY DISTRIBUTION
    print("\n7. YEARLY DISTRIBUTION:")
    year_counts = df['year'].value_counts().sort_index()
    print(f"   Articles by year:")
    for year, count in year_counts.items():
        print(f"     {year}: {count} articles")
    
    return {
        'exact_duplicates': exact_dupe_count,
        'title_year_duplicates': title_year_dupe_count,
        'edition_variants': edition_variants,
        # 'similar_titles': similar_titles,
        # 'wire_articles': wire_articles
    }

def find_similar_titles(df, similarity_threshold=0.9):
    """Find titles that are very similar (potential typos)"""
    titles = df['title'].dropna().unique()
    similar_pairs = []
    
    for i, title1 in enumerate(titles):
        for title2 in titles[i+1:]:
            similarity = SequenceMatcher(None, title1.lower(), title2.lower()).ratio()
            if similarity >= similarity_threshold and similarity < 1.0:
                similar_pairs.append((title1, title2, similarity))
    
    return sorted(similar_pairs, key=lambda x: x[2], reverse=True)

def find_wire_service_articles(df, text_similarity_threshold=0.85):
    """Find articles with similar text across different publications"""
    # Group by title to find potential wire articles
    title_groups = df.groupby('title')
    wire_articles = []
    
    for title, group in title_groups:
        if len(group) > 1:  # Multiple articles with same title
            unique_pubs = group['publication_name'].nunique()
            if unique_pubs > 1:  # Across different publications
                # Check if text is similar
                texts = group['main_text'].dropna()
                if len(texts) > 1:
                    # Compare first two texts as sample
                    text1, text2 = list(texts)[:2]
                    if len(text1) > 100 and len(text2) > 100:  # Meaningful text length
                        similarity = SequenceMatcher(None, text1[:1000], text2[:1000]).ratio()
                        if similarity >= text_similarity_threshold:
                            wire_articles.append(group.index.tolist())
    
    return wire_articles

def create_deduplication_strategy(df, analysis_results):
    """Suggest deduplication strategies based on analysis"""
    print("\n=== DEDUPLICATION RECOMMENDATIONS ===")
    
    exact_dupes = analysis_results['exact_duplicates']
    title_year_dupes = analysis_results['title_year_duplicates']
    edition_variants = analysis_results['edition_variants']
    
    total_potential_dupes = exact_dupes + title_year_dupes + edition_variants
    
    print(f"Total potential duplicates: {total_potential_dupes}")
    print(f"Potential final dataset size: {len(df) - total_potential_dupes}")
    
    print("\nRECOMMENDED DEDUPLICATION STEPS:")
    print("1. Remove exact duplicates (safe to remove)")
    print("2. For same title+year: keep one per publication")
    print("3. For edition variants: keep the most complete version")
    print("4. For wire articles: decide if you want multiple publications or just one")
    
    return total_potential_dupes

def remove_duplicates(df, strategy='conservative'):
    """
    Remove duplicates based on strategy
    Conservative: Only remove exact duplicates
    Moderate: Remove exact + same title/year/publication 
    Aggressive: Remove exact + same title/year (keep only one per title/year)
    """
    original_count = len(df)
    print(f"\nApplying '{strategy}' deduplication strategy...")
    
    if strategy == 'conservative':
        # Only remove exact duplicates
        df_clean = df.drop_duplicates(subset=['title', 'main_text', 'publication_name', 'year'])
        
    elif strategy == 'moderate':
        # Remove exact duplicates + same title/year/publication
        df_clean = df.drop_duplicates(subset=['title', 'year', 'publication_name'])
        
    elif strategy == 'aggressive':
        # Keep only one article per title/year combination
        df_clean = df.drop_duplicates(subset=['title', 'year'])
        
    else:
        print("Unknown strategy. Using conservative.")
        df_clean = df.drop_duplicates(subset=['title', 'main_text', 'publication_name', 'year'])
    
    removed_count = original_count - len(df_clean)
    print(f"Removed {removed_count} duplicates ({100*removed_count/original_count:.1f}%)")
    print(f"Final dataset: {len(df_clean)} articles")
    
    return df_clean

# Main execution function
def full_duplicate_analysis(csv_file="essential_newspaper_data.csv"):
    """Complete duplicate analysis workflow"""
    print("Loading data for duplicate analysis...")
    
    try:
        df = pd.read_csv(csv_file)
        print(f"Loaded {len(df)} articles")
    except Exception as e:
        print(f"Error loading {csv_file}: {e}")
        return None
    
    # Run analysis
    analysis_results = analyze_duplicates(df)
    
    # Create recommendations
    create_deduplication_strategy(df, analysis_results)
    
    # Show sample deduplication results
    print("\n=== SAMPLE DEDUPLICATION RESULTS ===")
    strategies = ['conservative', 'moderate', 'aggressive']
    
    for strategy in strategies:
        df_duplicate_removed = remove_duplicates(df.copy(), strategy=strategy)
    
    return df_duplicate_removed, analysis_results


In [5]:
# Usage
if __name__ == "__main__":
    df_duplicate_removed, results = full_duplicate_analysis()

Loading data for duplicate analysis...
Loaded 173999 articles
=== DUPLICATE ANALYSIS ===
Total articles: 173999

1. EXACT DUPLICATES:
   Exact duplicates: 11283
   Sample exact duplicates:
 year                                                                    title                       publication_name
20 17                                          IN BRIEF [Corrected 06/04/2017]         Chicago Tribune; Chicago, Ill.
200 5 GOPHERS UPDATE ; Gophers aiming for sweep at Ohio State: [METRO Edition]       Star Tribune; Minneapolis, Minn.
 2002                'A Season on the Brink' Creators Cry Foul: [HOME EDITION] Los Angeles Times; Los Angeles, Calif.

2. SAME TITLE + YEAR (different editions/publications):
   Articles with same title+year: 37035
   Sample same title+year:
 year                                                                    title                       publication_name
20 17                                          IN BRIEF [Corrected 06/04/2017]         Chicago Tr

In [6]:
len(df_duplicate_removed)  # Check the length of the final deduplicated DataFrame

149942

In [7]:
# Save the cleaned dataframe to a new CSV file
output_filename = "cleaned_newspaper_data.csv"
df_duplicate_removed.to_csv(output_filename, index=False, encoding='utf-8')

print(f"✅ Cleaned dataset saved to: {output_filename}")
print(f"📊 Dataset summary:")
print(f"   - Original articles: 173,999")
print(f"   - Duplicates removed: 24,057")
print(f"   - Final articles: 149,942")
print(f"   - Reduction: {100 * 24057 / 173999:.1f}%")

# Check file size
import os
file_size_mb = os.path.getsize(output_filename) / (1024**2)
print(f"   - File size: {file_size_mb:.2f} MB")

# Quick verification of the cleaned data
print(f"\n📋 Quick verification:")
print(f"   - Columns: {list(df_duplicate_removed.columns)}")
print(f"   - Year range: {df_duplicate_removed['year'].min()} to {df_duplicate_removed['year'].max()}")
print(f"   - Unique publications: {df_duplicate_removed['publication_name'].nunique()}")

# Show sample of cleaned data
print(f"\n📝 Sample of cleaned data:")
sample = df_duplicate_removed[['year', 'title', 'publication_name']].head(3)
print(sample.to_string(index=False))

✅ Cleaned dataset saved to: cleaned_newspaper_data.csv
📊 Dataset summary:
   - Original articles: 173,999
   - Duplicates removed: 24,057
   - Final articles: 149,942
   - Reduction: 13.8%
   - File size: 673.47 MB

📋 Quick verification:
   - Columns: ['year', 'date', 'title', 'main_text', 'publication_name', 'title_clean']
   - Year range: 20 02 to 2025
   - Unique publications: 248

📝 Sample of cleaned data:
 year                                                                                                                               title                              publication_name
20 02 A Sudden Tragedy, a Long Recovery; After Illness and Amputations, Virginia Tech's DuBose Discovers a World of Help: [FINAL Edition]         The Washington Post; Washington, D.C.
20 02                         Down-home coach leads La. Tech ; Barmore's program short on funds but not short on success: [FINAL Edition]                        USA TODAY; McLean, Va.
20 02                             

In [8]:
# Create a cleaning summary report
from datetime import datetime

summary_report = f"""
NEWSPAPER DATA CLEANING REPORT
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

ORIGINAL DATASET:
- Total articles: 173,999
- Years covered: {df_duplicate_removed['year'].min()} - {df_duplicate_removed['year'].max()}

DUPLICATES REMOVED:
- Exact duplicates: [number from your analysis]
- Same title + year: [number from your analysis] 
- Edition variants: [number from your analysis]
- Total removed: 24,057

FINAL CLEAN DATASET:
- Total articles: 149,942
- Reduction: 13.8%
- Unique publications: {df_duplicate_removed['publication_name'].nunique()}
- File: cleaned_newspaper_data.csv
- Size: {file_size_mb:.2f} MB

YEARLY DISTRIBUTION:
{df_duplicate_removed['year'].value_counts().sort_index().to_string()}
"""

with open("cleaning_report.txt", "w") as f:
    f.write(summary_report)

print("📄 Detailed report saved as: cleaning_report.txt")

📄 Detailed report saved as: cleaning_report.txt


##### **© 2024–2025 MD Rafsun Sheikh**
##### **Licensed under the Apache License, Version 2.0.**